In [1]:
# Cell 1: E-commerce SQL Analysis using your uploaded CSV
print("E-COMMERCE FUNNEL ANALYSIS - SQL ON CSV DATA")
print("=" * 60)

import pandas as pd
import sqlite3
from io import StringIO

# Load your uploaded CSV file
df = pd.read_csv('/content/sample_data/complete_ecommerce_analysis.csv')
print(f"CSV loaded: {len(df):,} records")
print(f"Columns: {len(df.columns)} features")
print("\nFirst 3 rows:")
display(df.head(3))

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Load your CSV data into SQL database
df.to_sql('ecommerce_data', conn, index=False, if_exists='replace')
print("\nCSV data loaded into SQL database!")

# Query 1: Revenue by Country & Device
print("\n" + "="*50)
print("REVENUE BY COUNTRY & DEVICE")
print("="*50)

query1 = """
SELECT
    country,
    device_type,
    SUM(total_amount) as total_revenue,
    COUNT(DISTINCT user_id) as customers,
    ROUND(AVG(total_amount), 2) as avg_order_value
FROM ecommerce_data
WHERE order_status = 'completed'
GROUP BY country, device_type
ORDER BY total_revenue DESC;
"""

result1 = pd.read_sql_query(query1, conn)
display(result1)

# Query 2: Conversion Funnel Analysis
print("\n" + "="*50)
print("CONVERSION FUNNEL ANALYSIS")
print("="*50)

query2 = """
SELECT
    COUNT(DISTINCT user_id) as total_users,
    COUNT(DISTINCT CASE WHEN total_sessions > 0 THEN user_id END) as users_with_sessions,
    COUNT(DISTINCT CASE WHEN add_to_cart_count > 0 THEN user_id END) as users_added_to_cart,
    COUNT(DISTINCT CASE WHEN order_status = 'completed' THEN user_id END) as users_purchased,

    -- Conversion rates
    ROUND(COUNT(DISTINCT CASE WHEN total_sessions > 0 THEN user_id END) * 100.0 /
          COUNT(DISTINCT user_id), 2) as browse_conversion_rate,
    ROUND(COUNT(DISTINCT CASE WHEN add_to_cart_count > 0 THEN user_id END) * 100.0 /
          COUNT(DISTINCT CASE WHEN total_sessions > 0 THEN user_id END), 2) as cart_conversion_rate,
    ROUND(COUNT(DISTINCT CASE WHEN order_status = 'completed' THEN user_id END) * 100.0 /
          COUNT(DISTINCT CASE WHEN add_to_cart_count > 0 THEN user_id END), 2) as purchase_conversion_rate
FROM ecommerce_data;
"""

result2 = pd.read_sql_query(query2, conn)
display(result2)

# Query 3: Product Performance
print("\n" + "="*50)
print("TOP 10 PRODUCTS BY REVENUE")
print("="*50)

query3 = """
SELECT
    product_name,
    category,
    ROUND(AVG(price), 2) as avg_price,
    SUM(quantity) as total_units_sold,
    SUM(quantity * price) as total_revenue,
    COUNT(DISTINCT user_id) as unique_customers
FROM ecommerce_data
WHERE order_status = 'completed'
GROUP BY product_name, category
ORDER BY total_revenue DESC
LIMIT 10;
"""

result3 = pd.read_sql_query(query3, conn)
display(result3)

# Query 4: Monthly Revenue Trends
print("\n" + "="*50)
print("MONTHLY REVENUE TRENDS")
print("="*50)

query4 = """
SELECT
    SUBSTR(order_date, 1, 7) as month,
    COUNT(DISTINCT order_id) as monthly_orders,
    SUM(total_amount) as monthly_revenue,
    COUNT(DISTINCT user_id) as monthly_customers,
    ROUND(SUM(total_amount) / COUNT(DISTINCT order_id), 2) as avg_order_value
FROM ecommerce_data
WHERE order_status = 'completed'
GROUP BY SUBSTR(order_date, 1, 7)
ORDER BY month;
"""

result4 = pd.read_sql_query(query4, conn)
display(result4)

conn.close()

print("\nANALYSIS COMPLETE!")
print("Key insights generated for the Tableau dashboard")

E-COMMERCE FUNNEL ANALYSIS - SQL ON CSV DATA
CSV loaded: 25,000 records
Columns: 22 features

First 3 rows:


,order_item_id,order_id,product_id,quantity,unit_price,product_name,category,price,user_id,order_date,...,items_count,country,device_type,signup_date,customer_segment,total_sessions,avg_pages_per_session,avg_session_duration,total_actions,add_to_cart_count
0,OI000000,O05753,P017,2,129.35,Sustainable Product 17,Home,129.35,U02836,2024-07-20 00:00:00,...,4,Egypt,Desktop,2024-08-03 00:00:00,New,6.0,10.000000,9.411372,18.0,0.0
1,OI000001,O04715,P043,2,168.20,Sustainable Product 43,Home,168.20,U06005,2024-05-19 00:00:00,...,7,UAE,Tablet,2024-04-02 00:00:00,New,6.0,7.333333,7.585114,20.0,0.0
2,OI000002,O07003,P012,2,77.76,Sustainable Product 12,Home,77.76,U08596,2024-11-21 00:00:00,...,1,KSA,Desktop,2024-09-07 00:00:00,New,4.0,9.250000,9.132250,15.0,1.0



CSV data loaded into SQL database!

REVENUE BY COUNTRY & DEVICE


,country,device_type,total_revenue,customers,avg_order_value
0,UAE,Mobile,6091011.99,1422,997.38
1,UAE,Desktop,2982508.70,689,1028.45
2,KSA,Mobile,2486759.48,565,1033.14
3,KSA,Desktop,1180734.69,253,1031.21
4,Egypt,Mobile,1071018.07,269,978.10
5,Qatar,Mobile,1003503.21,248,973.33
6,Oman,Mobile,604125.80,134,977.55
7,UAE,Tablet,575806.51,121,1026.39
8,Kuwait,Mobile,545170.49,134,1032.52
9,Egypt,Desktop,501064.03,119,976.73



CONVERSION FUNNEL ANALYSIS


...


TOP 10 PRODUCTS BY REVENUE


,product_name,category,avg_price,total_units_sold,total_revenue,unique_customers
0,Sustainable Product 44,Accessories,459.43,809,371678.87,392
1,Sustainable Product 49,Beauty,487.24,744,362506.56,352
2,Sustainable Product 35,Fashion,460.91,755,347987.05,366
3,Sustainable Product 39,Electronics,463.69,749,347303.81,367
4,Sustainable Product 13,Fashion,393.87,872,343454.64,413
5,Sustainable Product 34,Beauty,409.57,776,317826.32,368
6,Sustainable Product 1,Fashion,434.87,730,317455.10,341
7,Sustainable Product 46,Electronics,434.05,720,312516.00,341
8,Sustainable Product 38,Accessories,452.10,676,305619.60,333
9,Sustainable Product 9,Beauty,385.00,787,302995.00,359



MONTHLY REVENUE TRENDS


,month,monthly_orders,monthly_revenue,monthly_customers,avg_order_value
0,2024-01,475,1454730.27,467,3062.59
1,2024-02,465,1453935.74,457,3126.74
2,2024-03,474,1540758.53,460,3250.55
3,2024-04,450,1473988.35,440,3275.53
4,2024-05,509,1662695.04,493,3266.59
5,2024-06,482,1652939.79,469,3429.34
6,2024-07,468,1577732.50,460,3371.22
7,2024-08,484,1541450.11,470,3184.81
8,2024-09,468,1642283.39,458,3509.15
9,2024-10,494,1662720.87,481,3365.83



ANALYSIS COMPLETE!
Key insights generated for your Tableau dashboard
